<img src="http://cailab.appspot.com/images/logo.png" />

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

### Importar Imagenes de Emociones

In [ ]:
ira_dir = os.path.join('CK/anger')
contento_dir = os.path.join('CK/contempt')
disgusto_dir = os.path.join('CK/disgust')
miedo_dir = os.path.join('CK/fear')
feliz_dir = os.path.join('CK/happy')
tristeza_dir = os.path.join('CK/sadness')
sorpresa_dir = os.path.join('CK/surprise')

ira_archivos = os.listdir(ira_dir)
contento_archivos = os.listdir(contento_dir)
disgusto_archivos = os.listdir(disgusto_dir)
miedo_archivos = os.listdir(miedo_dir)
feliz_archivos = os.listdir(feliz_dir)
tristeza_archivos = os.listdir(tristeza_dir)
sorpresa_archivos = os.listdir(sorpresa_dir)

### Deplegar 2 Images de DataSet

In [ ]:
total = 2

# cargar las rutas de 5 imagenes
ira_ejemplos = [os.path.join(ira_dir, nombre) for nombre in ira_files[1:total+1]]

# deplegar las 2 imagenes
for i, ruta in enumerate(ira_ejemplos):
    img = mpimg.imread(ruta)
    plt.imshow(img)
    plt.axis('Off')
    plt.show()

### Tensorflow + Keras

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

#### Definir la configuracion de entrenamiento

In [ ]:
ENTRENAMIENTO_DIR = "CK/"

# configuracion de entrenamiento
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# generador
generador_entrenamiento = entrenamiento_datagen.flow_from_directory(
    ENTRENAMIENTO_DIR,
    target_size=(48,48),
    class_mode='categorical'
)

#### Crear la red neuronal convolutional (modelo)

In [ ]:
# Arquitectura convolucional

modelo = tf.keras.models.Sequential([
    # primera capa
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(48, 48, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # segunda capa
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # tercera capa
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # cuarta capa
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Red Neuronal Flat
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuronas ocultas
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

# Entrenar el modelo y salvarlo como reconcedor-facial.h5

modelo.summary()
modelo.compile(loss = 'categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])
# entrenar 300 veces el modelo (esto dura un rato. Vaya por cafe!)
historia = modelo.fit_generator(generador_entrenamiento, epochs = 300, verbose = 1)
modelo.save("reconcedor-facial.h5")

In [ ]:
# Entre mas cercano a 1, mejor es la capacidad predictiva.
acc = historia.history['acc']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Exactitud de la Predicción')
plt.title('Rendimiento')
plt.legend(loc=0)
plt.figure()

plt.show()

### Predicción de Emociones

In [ ]:
# cargamos el modelo desde el archivo
# para no tener que entrenarlo cada vez

from keras.preprocessing import image
import numpy as np

modelo = tf.keras.models.load_model('reconcedor-facial.h5')

In [ ]:
# prediccion

# classes
etiquetas = ['ira','contento','disgusto','miedo','feliz','tristeza','sorpresa']
archivo = feliz_dir + '/' + feliz_archivos[30]

img = image.load_img(archivo, target_size=(48, 48))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

imagenes = np.vstack([x])
clases = modelo.predict(imagenes, batch_size=1)
indice = clases.tolist()[0].index(1.0)

img = mpimg.imread(archivo)
plt.imshow(img)
plt.axis('Off')
plt.show()

print("Emocion estimada:", etiquetas[indice],"!")